In [ ]:
# This book takes weather data from Mason, Cherokee, and Le Claire weather stations (all Iowa)
# Precipitation and Mid-range temperature are averaged as a proxy for Iowa weather

# These features are engineered to be described as weekly deviation from
# the historical average for a week in that particular month

# Month number is also added as a feature

# All features from months November - March are set to zero because no corn is in the ground then
# Month numbers are adjusted accordingly so that month 1 is April

In [29]:
import pandas as pd

In [30]:
# Magic numbers
NO_STATIONS = 3

In [31]:
# Data URLs
mason = 'https://raw.githubusercontent.com/ptraver/data/main/mason_full_D_1960.csv'
cherokee = 'https://raw.githubusercontent.com/ptraver/data/main/cherokee_full_D_1960.csv'
le_claire = 'https://raw.githubusercontent.com/ptraver/data/main/le_claire_full_D_1960.csv'

In [60]:
# Bring data to dataframes
mason_data = pd.read_csv(mason, parse_dates=True, index_col='DATE', dayfirst=True)
cherokee_data = pd.read_csv(cherokee, parse_dates=True, index_col='DATE', dayfirst=True)
le_claire_data = pd.read_csv(le_claire, parse_dates=True, index_col='DATE', dayfirst=True)

In [61]:
# drop columns
mason_data = mason_data.drop(['STATION', 'SNOW', 'SNWD'], axis=1)
cherokee_data = cherokee_data.drop(['STATION', 'SNOW', 'NAME', 'SNWD'], axis=1)
le_claire_data = le_claire_data.drop(['STATION', 'SNOW', 'NAME', 'SNWD'], axis=1)

In [62]:
# filter dates
mason_data = mason_data['1974-01-01':'2017-12-31']
cherokee_data = cherokee_data['1974-01-01':'2017-12-31']
le_claire_data = le_claire_data['1974-01-01':'2017-12-31']

In [63]:
# check missing values
print(mason_data.isnull().any())
print(cherokee_data.isnull().any())
print(le_claire_data.isnull().any())

PRCP     True
TMAX    False
TMIN    False
dtype: bool
PRCP    True
TMAX    True
TMIN    True
dtype: bool
PRCP    True
TMAX    True
TMIN    True
dtype: bool


In [64]:
# Check number of missing values
print(len(mason_data[mason_data['PRCP'].isnull()]))

print(len(cherokee_data[cherokee_data['PRCP'].isnull()]))
print(len(cherokee_data[cherokee_data['TMAX'].isnull()]))
print(len(cherokee_data[cherokee_data['TMIN'].isnull()]))

print(len(le_claire_data[le_claire_data['PRCP'].isnull()]))
print(len(le_claire_data[le_claire_data['TMAX'].isnull()]))
print(len(le_claire_data[le_claire_data['TMIN'].isnull()]))


3
9
12
14
382
20
22


In [65]:
# replace missing values with the last valid observation

mason_data.fillna(method='ffill', inplace=True)
cherokee_data.fillna(method='ffill', inplace=True)
le_claire_data.fillna(method='ffill', inplace=True)

In [66]:
# for each station make (tmax + tmin) / 2 ....mid_range

mason_data['T_MIDRANGE'] = (mason_data['TMAX'] + mason_data['TMIN']) / 2
cherokee_data['T_MIDRANGE'] = (cherokee_data['TMAX'] + cherokee_data['TMIN']) / 2
le_claire_data['T_MIDRANGE'] = (le_claire_data['TMAX'] + le_claire_data['TMIN']) / 2

In [67]:
# drop max and min temp
mason_data = mason_data.drop(['TMAX', 'TMIN'], axis=1)
cherokee_data = cherokee_data.drop(['TMAX', 'TMIN'], axis=1)
le_claire_data = le_claire_data.drop(['TMAX', 'TMIN'], axis=1)

In [68]:
# combine weather stations to iowa_weather

iowa_weather = mason_data.add(cherokee_data)
iowa_weather = iowa_weather.add(le_claire_data)

In [69]:
iowa_weather.head()

,PRCP,T_MIDRANGE
DATE,,
1974-01-01,0.0,-69.15
1974-01-02,1.0,-55.30
1974-01-03,1.5,-51.15
1974-01-04,0.0,-46.70
1974-01-05,0.0,-40.55


In [70]:
# divide by 3 for readable average

iowa_weather['PRCP'] = (iowa_weather['PRCP'] / NO_STATIONS).round(2)
iowa_weather['T_MIDRANGE'] = (iowa_weather['T_MIDRANGE'] / NO_STATIONS).round(2)

iowa_weather.head()

,PRCP,T_MIDRANGE
DATE,,
1974-01-01,0.00,-23.05
1974-01-02,0.33,-18.43
1974-01-03,0.50,-17.05
1974-01-04,0.00,-15.57
1974-01-05,0.00,-13.52


In [71]:
# NEXT THREE CELLS FOR FEATURE ENGINEERING

# Find average daily figures by week in month

#resample to weekly
weekly_iowa_weather = iowa_weather.resample('W').mean()

#resample to monthly
monthly_iowa_weather = weekly_iowa_weather.resample('M').mean()

#add month column
monthly_iowa_weather['MONTH'] = monthly_iowa_weather.index.month

monthly_iowa_weather

,PRCP,T_MIDRANGE,MONTH
DATE,,,
1974-01-31,0.917262,-9.965714,1
1974-02-28,0.645357,-4.837500,2
1974-03-31,1.408857,1.306000,3
1974-04-30,2.389643,9.790357,4
1974-05-31,4.951786,13.290357,5
...,...,...,...
2017-08-31,2.825357,20.007857,8
2017-09-30,1.548929,19.411071,9
2017-10-31,4.021714,12.935429,10


In [72]:
#get daily mean precipitation by week in month
avg_daily_prcp_by_week_in_month = monthly_iowa_weather.groupby('MONTH').agg(
    AVG_MTH_PRCP =pd.NamedAgg(column='PRCP', aggfunc='mean')
    )

avg_daily_prcp_by_week_in_month

,AVG_MTH_PRCP
MONTH,
1,0.799543
2,0.945894
3,1.632602
4,2.919914
5,3.398539
6,4.084391
7,3.430543
8,3.548078
9,2.765602


In [73]:
#get daily mean t_midrange by week in month
avg_daily_t_midrange_by_week_in_month = monthly_iowa_weather.groupby('MONTH').agg(
    AVG_MTH_T_MIDRANGE =pd.NamedAgg(column='T_MIDRANGE', aggfunc='mean')
    )

avg_daily_t_midrange_by_week_in_month

,AVG_MTH_T_MIDRANGE
MONTH,
1,-7.692177
2,-5.685621
3,0.702737
4,8.571718
5,14.817023
6,20.631394
7,23.203263
8,21.904114
9,18.005638


In [75]:
# BACK TO MAIN DATASET

# resample to week
iowa_weather = iowa_weather.resample('W').mean()

In [76]:
# add month number
iowa_weather['MONTH'] = iowa_weather.index.month

In [77]:
iowa_weather.head()

,PRCP,T_MIDRANGE,MONTH
DATE,,,
1974-01-06,0.393333,-17.150000,1
1974-01-13,0.627143,-19.395714,1
1974-01-20,0.424286,-1.230000,1
1974-01-27,2.224286,-2.087143,1
1974-02-03,0.075714,-5.504286,2


In [78]:
# add average daily prcp by week in month
iowa_weather = pd.merge(iowa_weather, avg_daily_prcp_by_week_in_month, on='MONTH', how='left')
iowa_weather.index = weekly_iowa_weather.index
iowa_weather.head()

,PRCP,T_MIDRANGE,MONTH,AVG_MTH_PRCP
DATE,,,,
1974-01-06,0.393333,-17.150000,1,0.799543
1974-01-13,0.627143,-19.395714,1,0.799543
1974-01-20,0.424286,-1.230000,1,0.799543
1974-01-27,2.224286,-2.087143,1,0.799543
1974-02-03,0.075714,-5.504286,2,0.945894


In [79]:
# add average daily t_midrange by week in month
iowa_weather = pd.merge(iowa_weather, avg_daily_t_midrange_by_week_in_month, on='MONTH', how='left')
iowa_weather.index = weekly_iowa_weather.index
iowa_weather.head()

,PRCP,T_MIDRANGE,MONTH,AVG_MTH_PRCP,AVG_MTH_T_MIDRANGE
DATE,,,,,
1974-01-06,0.393333,-17.150000,1,0.799543,-7.692177
1974-01-13,0.627143,-19.395714,1,0.799543,-7.692177
1974-01-20,0.424286,-1.230000,1,0.799543,-7.692177
1974-01-27,2.224286,-2.087143,1,0.799543,-7.692177
1974-02-03,0.075714,-5.504286,2,0.945894,-5.685621


In [80]:
# subtract daily mean by week in month from each value
iowa_weather['PRCP'] = iowa_weather['PRCP'] - iowa_weather['AVG_MTH_PRCP']
iowa_weather['T_MIDRANGE'] = iowa_weather['T_MIDRANGE'] -iowa_weather['AVG_MTH_T_MIDRANGE']

iowa_weather.head()

,PRCP,T_MIDRANGE,MONTH,AVG_MTH_PRCP,AVG_MTH_T_MIDRANGE
DATE,,,,,
1974-01-06,-0.406210,-9.457823,1,0.799543,-7.692177
1974-01-13,-0.172400,-11.703537,1,0.799543,-7.692177
1974-01-20,-0.375258,6.462177,1,0.799543,-7.692177
1974-01-27,1.424742,5.605034,1,0.799543,-7.692177
1974-02-03,-0.870179,0.181335,2,0.945894,-5.685621


In [81]:
# Drop the columns won't need
# Month dropped here and added in again later
iowa_weather = iowa_weather.drop(['AVG_MTH_PRCP', 'AVG_MTH_T_MIDRANGE', 'MONTH'], axis=1)
iowa_weather.head()

,PRCP,T_MIDRANGE
DATE,,
1974-01-06,-0.406210,-9.457823
1974-01-13,-0.172400,-11.703537
1974-01-20,-0.375258,6.462177
1974-01-27,1.424742,5.605034
1974-02-03,-0.870179,0.181335


In [82]:
iowa_weather.tail()

,PRCP,T_MIDRANGE
DATE,,
2017-12-03,-1.206430,8.713833
2017-12-10,-1.106430,2.510976
2017-12-17,-1.177859,3.370976
2017-12-24,-0.925002,2.018119
2017-12-31,-0.175002,-12.254739


In [83]:
# Last two samples must be cut to keep the shape in line with price data
iowa_weather = iowa_weather[:-2]

In [84]:
# Split data
from sklearn.model_selection import train_test_split

weather_train, out_of_sample = train_test_split(iowa_weather, test_size=0.2, shuffle=False)
weather_valid, weather_test = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

In [85]:
# Scale data
from sklearn.preprocessing import StandardScaler

weather_scaler = StandardScaler()

train_scaled = weather_scaler.fit_transform(weather_train)
valid_scaled = weather_scaler.transform(weather_valid)
test_scaled = weather_scaler.transform(weather_test)

In [86]:
iowa_weather_train = pd.DataFrame(train_scaled, columns=weather_train.columns, index=weather_train.index)
iowa_weather_valid = pd.DataFrame(valid_scaled, columns=weather_valid.columns, index=weather_valid.index)
iowa_weather_test = pd.DataFrame(test_scaled, columns=weather_test.columns, index=weather_test.index)

In [87]:
iowa_weather_train.head()

,PRCP,T_MIDRANGE
DATE,,
1974-01-06,-0.151911,-2.287060
1974-01-13,-0.048815,-2.834092
1974-01-20,-0.138263,1.590880
1974-01-27,0.655428,1.382090
1974-02-03,-0.356494,0.060935


In [88]:
# Re-add month number
iowa_weather_train['MONTH'] = iowa_weather_train.index.month
iowa_weather_valid['MONTH'] = iowa_weather_valid.index.month
iowa_weather_test['MONTH'] = iowa_weather_test.index.month

In [89]:
# process weather around growing season

# Set all variables for months 11,12,1,2,3 to zero

# For all 'MONTH' values, scale to between 0 and 1

iowa_weather_train = iowa_weather_train.apply(lambda x: x-3 if x.name == 'MONTH' else x)
iowa_weather_train[iowa_weather_train['MONTH'].isin([8, 9, -2, -1, 0])] = 0
iowa_weather_train = iowa_weather_train.apply(lambda x: x/7 if x.name == 'MONTH' else x)

iowa_weather_valid = iowa_weather_valid.apply(lambda x: x-3 if x.name == 'MONTH' else x)
iowa_weather_valid[iowa_weather_valid['MONTH'].isin([8, 9, -2, -1, 0])] = 0
iowa_weather_valid = iowa_weather_valid.apply(lambda x: x/7 if x.name == 'MONTH' else x)

iowa_weather_test = iowa_weather_test.apply(lambda x: x-3 if x.name == 'MONTH' else x)
iowa_weather_test[iowa_weather_test['MONTH'].isin([8, 9, -2, -1, 0])] = 0
iowa_weather_test = iowa_weather_test.apply(lambda x: x/7 if x.name == 'MONTH' else x)

iowa_weather_train.head(50)

,PRCP,T_MIDRANGE,MONTH
DATE,,,
1974-01-06,0.000000,0.000000,0.000000
1974-01-13,0.000000,0.000000,0.000000
1974-01-20,0.000000,0.000000,0.000000
1974-01-27,0.000000,0.000000,0.000000
1974-02-03,0.000000,0.000000,0.000000
1974-02-10,0.000000,0.000000,0.000000
1974-02-17,0.000000,0.000000,0.000000
1974-02-24,0.000000,0.000000,0.000000
1974-03-03,0.000000,0.000000,0.000000


In [90]:
iowa_weather_train.to_csv('iowa_weather_train.csv')
iowa_weather_valid.to_csv('iowa_weather_valid.csv')
iowa_weather_test.to_csv('iowa_weather_test.csv')

# End